# Рекомендация тарифов

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. 

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
def review(df): 
    """функция для отображения общей информации"""
    info = display(df.head(10)), df.info(), display(df.describe())

## Откроем и изучим файл

In [5]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [6]:
review(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Приведем тип данных в столбцах `calls` и `messages` к `int64` и проверим данные на явные дубликаты.

In [7]:
for column in df[['calls','messages']]:
    df[column] = df[column].astype("int")

In [8]:
df.duplicated().sum()

0

In [9]:
review(df)

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0
5,58,344.56,21,15823.37,0
6,57,431.64,20,3738.90,1
7,15,132.40,6,21911.60,0
8,7,43.39,3,2538.67,1
9,90,665.41,38,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Вывод

В целом данные чистые, была лишь произведена замена типа данных в `calls` и `messages` на `int64`.

## Разобьем данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [10]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Спрятанной тестовой выборки нет. Поэтому, данные разобьем на три части: обучающую, валидационную и тестовую. Исходные данные разбивают в соотношении 3:1:1.

### Обучающая

In [11]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.4, 
                                                                            random_state=12345
                                                                           ) 

Размер выборки обучающей выборки

In [12]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


### Валидационная и тестовая

In [13]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345
                                                                           ) 

Размер валидационной выборки

In [14]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


Размер тестовой выборки

In [15]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


## Исследуем модели

### `Decision Tree Classifier`

In [16]:
best_model_dtc = None
best_depth_dtc = 0
best_result_dtc = 0
for depth in tqdm(range(1, 11)):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth = depth).fit(features_train, target_train)
    predictions_valid_dtc = model_dtc.predict(features_valid)
    result_dtc = accuracy_score(predictions_valid_dtc, target_valid)
    if best_result_dtc < result_dtc:
     best_model_dtc = model_dtc
     best_depth_dtc = depth
     best_result_dtc = result_dtc

100%|██████████| 10/10 [00:00<00:00, 211.53it/s]


In [17]:
print('best max_depth =', depth, ': ', 'best Accuracy = ', best_result_dtc, end ='') 

best max_depth = 10 :  best Accuracy =  0.7744945567651633

#### Вывод

`Accuracy` лучшей модели из десяти равно `0.7744945567651633`, с гиперпараметром `max_depth` равным - `10`, ресурсозатратность же равна, при заданных `10` иттераций, равна 172.03ит/сек.

### `RandomForestClassifier`

In [18]:
best_model_rfc = None
best_est_rfc = 0
best_depth_rfc = 0
best_result_rfc = 0
for est in tqdm(range(1, 11)):
    for depth in range(1, 11):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth).fit(features_train, target_train)
        predictions_valid_rfc = model_rfc.predict(features_valid)
        accuracy_rfc = accuracy_score(predictions_valid_rfc, target_valid)
        if best_result_rfc < result_rfc:
         best_model_rfc = model_rfc
         best_est_rfc = est
         best_depth_rfc = depth
         best_result_rfc = result_rfc

100%|██████████| 10/10 [00:01<00:00,  7.57it/s]


In [19]:
print('best n_estimators =', est, ': ', 'best max_depth =', depth, ': ', 'best Accuracy = ', best_result_rfc, end ='') 

best n_estimators = 10 :  best max_depth = 10 :  best Accuracy =  0.7916018662519441

#### Вывод

`Accuracy` лучшей модели из десяти равно `0.7916018662519441`, с гиперпараметром `n_estimators` равным - `10` и `max_depth` равным `10`, ресурсозатратность же равна, при заданных `10` иттераций, равна 6.04ит/сек.

### `Logistic Regression`

In [20]:
model_lr = LogisticRegression(random_state=12345).fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)
print('Accuracy LogisticRegression:', accuracy_lr, end ='')

Accuracy LogisticRegression: 0.7589424572317263

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Вывод

Accuracy модели `LogisticRegression` равно `0.7589424572317263`

### Вывод

- наилучший результат `Accuracy` у модели `решающего дерева` - `DecisionTreeClassifier` - `Accuracy` равен `0.7744945567651633`, ресурсозатратность же при этом меньшая - при заданных `10` иттераций, равна 172.03ит/сек;
- наилучший результат `Accuracy` у модели `случайного леса` - `RandomForestClassifier` - `0.7916018662519441`, ресурсозатратность же, при заданных `10` иттераций, равна 6.04ит/сек.
- результат `Accuracy` у модели `логистической регрессии` - `LogisticRegression` - `0.7589424572317263`.

## Проверим модель на тестовой выборке

Наиболее эффективно (точность/скорость) себя показала модель `случайного леса` - `RandomForestClassifier`, поэтому проверим ее на тестовой выборке

In [21]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators = 10, max_depth = 10).fit(features_train, target_train)
predictions_test_rfc = model_rfc.predict(features_test)
result_test_rfc = accuracy_score(predictions_test_rfc, target_test)
print('Accuracy RandomForestClassifier:', result_test_rfc)

Accuracy RandomForestClassifier: 0.8009331259720062


### Вывод

`Accuracy` у модели `случайного леса` - `RandomForestClassifier` на тестовой выборке равна `0.8009331259720062`.

## Проверим модель на адекватность

Для проверки используем модель `DummyClassifier`.

In [22]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dc)

Accuracy DummyClassifier: 0.7060653188180405


In [23]:
print('Accuracy RandomForestClassifier:', result_test_rfc)

Accuracy RandomForestClassifier: 0.8009331259720062


### Вывод

`Accuracy` модели `DummyClassifier` ниже, чем у модели `RandomForestClassifier` - модель адекватна и эффективна.

## Общий вывод

- предоставленные данные чистые, без артефактов и прочего;
- были исследованы модели `Decision Tree Classifier`, `RandomForestClassifier` и `Logistic Regression`;
- наилучшую эффективность (точность/скорость) показала модель `RandomForestClassifier` с гиперпараметрами `n_estimators` равным - `10` и `max_depth` равным `10`;
- модель `RandomForestClassifier` с наилучшими гиперапараметрами была проверена на тестовой выборке и показала результат `Accuracy` - `0.8009331259720062`;
- также модель `RandomForestClassifier` проверенная на тестовой выборке была проверена на вменяемость - для проверки использовалась модель `DummyClassifier`, которая показала результат `Accuracy` - `0.7060653188180405`.